This notebook has the following changes compared to "Create_wav_files_for_one_tone.nb"

1. Frequencies are distributed with a 20*log(1003/1000) difference in the log domain. 
2. Background tones are chosen from extremes i.e. between [100,-2low_sigma+low_mean] and between [2high_sigma+high_mean,2000].
3. Difficult test tones (those with 2 or more background tones) have 1-3 simple trials between them
4. Tones are enveloped.

In [1]:
import math
import wave
import struct
import pdb
import numpy as np
import matplotlib.pyplot as plt
import csv
import scipy.stats
import random

In [2]:
"""
Variable audio will contain a long list of samples (i.e. floating point numbers describing the
waveform).  If you were working with a very long sound you'd want to stream this to
disk instead of buffering it all in memory list this.  But most sounds will fit in 
memory.   
"""

def append_silence(audio, sample_rate, duration_milliseconds=250):
    """
    Adding silence is easy - we add zeros to the end of our array
    """
    num_samples = duration_milliseconds * (sample_rate / 1000.0)

    for x in range(int(num_samples)): 
        audio.append(0.0)
    return audio

def append_sinewave(audio, sample_rate, 
                    freq, 
                    duration_milliseconds=400, 
                    volume=1.0):
    """
    The sine wave generated here is the standard beep.  If you want something
    more aggresive you could try a square or saw tooth waveform.   Though there
    are some rather complicated issues with making high quality square and
    sawtooth waves... which we won't address here
    """ 
    num_samples = duration_milliseconds * (sample_rate / 1000.0)
    signal = []
    
    for x in range(int(num_samples)):
        signal.append(volume * math.sin(2 * math.pi * freq * (x/sample_rate)))
    audio.append(envelopeKCW(signal, 20, sample_rate))
    return audio

def envelopeKCW(signal,rampDur,sample_rate):
    """
    This function tries to remove the transients in the signal by enveloping the first and last period.
    Ramp duration is defined by rampDur in **ms** envelope(signal,rampDuration,samplerate)
    """
    samples = int(np.round((rampDur/1000)*sample_rate))
    x = np.arange(-np.pi,0,np.pi/samples)
    y = np.arange(0,np.pi,np.pi/samples)
    output_signal = np.copy(signal)

    # Prepare the envelope functions
    envelope_function = np.cos(x)/2+0.5;
    
    # Fade in
    for i in range(samples):
        output_signal[i] = signal[i] * envelope_function[i];

    # Fade out
    for i in range(samples-1):
        current_position = len(signal) - i - 1;
        output_signal[current_position] = signal[current_position] * envelope_function[i+1];
    return output_signal


def save_wav(audio, sample_rate, file_name):
    # Open up a wav file
    wav_file=wave.open(file_name,"w")

    # wav params
    nchannels = 1
    sampwidth = 2

    # 44100 is the industry standard sample rate - CD quality.  If you need to
    # save on file size you can adjust it downwards. The standard for low quality
    # is 8000 or 8kHz.
    nframes = len(audio)
    comptype = "NONE"
    compname = "not compressed"
    wav_file.setparams((nchannels, sampwidth, sample_rate, nframes, comptype, compname))

    # WAV files here are using short, 16 bit, signed integers for the 
    # sample size.  So we multiply the floating point data we have by 32767, the
    # maximum value for a short integer.  NOTE: It is theortically possible to
    # use the floating point -1.0 to 1.0 data directly in a WAV file but not
    # obvious how to do that using the wave module in python.
    for sample in audio:
        wav_file.writeframes(struct.pack('h', int( sample * 32767.0 )))
        
    wav_file.close()
    return

def generate_tone_cloud(freq_list):
    audio = []
    sample_rate = 44100.0   
    
    freq_0 = freq_list[0]; freq_1 = freq_list[1];
    audio = append_sinewave(audio = audio, sample_rate = sample_rate, freq = freq_0)
    audio = append_silence(audio = audio, sample_rate = sample_rate)
    audio = append_sinewave(audio = audio, sample_rate = sample_rate, freq = freq_1)
    audio = append_silence(audio = audio, sample_rate = sample_rate)
    return audio, sample_rate

In [4]:
def train_task(n_trials = 10, n_tones = 3, p_low = 0.5, p_back = 0.2):
    freq_seq = np.arange(100,2000,3) #array of possible true tones
    log_freq_seq_limits = [np.log10(freq_seq[0]), np.log10(freq_seq[-1])]
    log_freq_seq_mid = np.mean(log_freq_seq_limits)
    log_freq_low = [log_freq_seq_mid - 0.2,0.1]  #low freq condition is gaussian 
    log_freq_high = [log_freq_seq_mid + 0.2,0.1] #high freq condition is gaussian
    trial_tones = []
    dist_chosen = []

    sample_x = np.linspace(log_freq_seq_limits[0], log_freq_seq_limits[1], 101);

    lowDistWeights = scipy.stats.norm.pdf(loc = log_freq_low[0], scale = log_freq_low[1], x = sample_x);
    lowDistWeights = lowDistWeights/sum(lowDistWeights);
    highDistWeights = scipy.stats.norm.pdf(loc = log_freq_high[0], scale = log_freq_high[1], x = sample_x);
    highDistWeights = highDistWeights/sum(highDistWeights);

    bgDistWeights = scipy.stats.uniform.pdf(loc = freq_seq[0], scale = freq_seq[-1], x = 10**sample_x);
    bgDistWeights = bgDistWeights/sum(bgDistWeights);

    plt.figure()
    plt.plot(sample_x,lowDistWeights) 
    plt.plot(sample_x,highDistWeights)
    plt.plot(sample_x,bgDistWeights)
    plt.show()
    
    for trial in range(n_trials):
        signal_rand = np.random.random()
        low_dist = signal_rand < p_low #choosing true tone from either low or high condition
        tones = []
        for n_tone in range(n_tones):
            signal_back = np.random.random()
            background = signal_back < p_back #choosing background or true tone
            if background:
                #background freq is chosen from a uniform distribution
                if np.random.random() < 0.5:
                    tone = float(np.random.uniform(log_freq_seq_limits[0], \
                                                        log_freq_seq_limits[0]+0.1, size = 1))
                else:
                    tone = float(np.random.uniform(log_freq_seq_limits[1], \
                                                        log_freq_seq_limits[1]-0.1, size = 1))
                nearest_tone = freq_seq[np.argmin(np.abs(freq_seq - 10**tone))]               
            else: 
                if low_dist:
                    tone = min(max(np.random.randn()*log_freq_low[1] + log_freq_low[0],\
                                   log_freq_low[0]-log_freq_low[1]),log_freq_low[0]+log_freq_low[1])
                    nearest_tone = freq_seq[np.argmin(np.abs(freq_seq - 10**tone))]
                else:
                    tone = min(max(np.random.randn()*log_freq_high[1] + log_freq_high[0],\
                                   log_freq_high[0]-log_freq_high[1]),log_freq_high[0]+log_freq_high[1])
                    nearest_tone = freq_seq[np.argmin(np.abs(freq_seq - 10**tone))]
            tones.append(nearest_tone)
        trial_tones.append(tones)
        dist_chosen.append(low_dist)
    return trial_tones, dist_chosen


In [ ]:
# This is for the training part of the three tones expt.
training_trials, dist_chosen = train_task(n_trials = 50, n_tones = 2)

cnt_low = 0; cnt_high = 0
rows_csv_tr = []
for i_trial in range(len(training_trials)):
    audio, sample_rate = generate_tone_cloud(training_trials[i_trial])    
    if dist_chosen[i_trial]:
        save_wav(audio, sample_rate, file_name = "soundfiles_two_tones/train_wave_low_"+str(cnt_low)+".wav")
        rows_csv_tr.append(["train_wave_low_"+str(cnt_low)+".wav", 'l', training_trials[i_trial]])
        cnt_low += 1 
    else:
        save_wav(audio, sample_rate, file_name = "soundfiles_two_tones/train_wave_high_"+str(cnt_high)+".wav")
        rows_csv_tr.append(["train_wave_high_"+str(cnt_high)+".wav", 'h', training_trials[i_trial]])
        cnt_high += 1 

In [ ]:
# field names  
fields = ['Name', 'corrAns', 'Tones']  
    
# name of csv file  
filename = "train_two_tones_files.csv"
    
# writing to csv file  
with open(filename, 'w') as csvfile:  
    # creating a csv writer object  
    csvwriter = csv.writer(csvfile)          
    # writing the fields  
    csvwriter.writerow(fields)          
    # writing the data rows  
    csvwriter.writerows(rows_csv_tr) 

In [62]:
def test_task(n_trials = 10, n_tones = 3, p_low = 0.5, p_back = 0.3):
    freq_seq = np.arange(100,2000,1) #array of possible true tones
    log_freq_seq_array = np.arange(np.log10(freq_seq[0]), np.log10(freq_seq[-1]), np.log10(1003/1000)*20)
    log_freq_seq_mid = np.median(log_freq_seq_array)
    log_freq_low = [log_freq_seq_mid - 0.2,0.1]  #low freq condition is gaussian 
    log_freq_high = [log_freq_seq_mid + 0.2,0.1] #high freq condition is gaussian
    trial_tones = []
    dist_chosen = []
    kind_of_tones = []
    
    for trial in range(n_trials):
        signal_rand = np.random.random()
        low_dist = signal_rand < p_low #choosing true tone from either low or high condition
        tones = []
        tone_kind = []
        for n_tone in range(n_tones):
            signal_back = np.random.random()
            background = signal_back < p_back #choosing background or true tone
            if background:
                if np.random.random() < 0.5:                    
                    tone = \
                    np.random.choice(log_freq_seq_array[0:np.argmin(np.abs(log_freq_seq_array -\
                                                        (log_freq_low[0]-2*log_freq_low[1])))])
                else:
                    tone = \
                    np.random.choice(log_freq_seq_array[np.argmin(np.abs(log_freq_seq_array -\
                                                        (log_freq_high[0]+2*log_freq_high[1]))):-1])
                nearest_tone = freq_seq[np.argmin(np.abs(freq_seq - 10**tone))] \
                #background freq is chosen from a uniform distribution
                tone_kind.append('b')
            else: 
                if low_dist:
                    tone = min(max(np.random.randn()*log_freq_low[1] + log_freq_low[0],\
                                   log_freq_seq_array[0]),log_freq_seq_array[-1])                    
                    tone_kind.append('l')
                else:
                    tone = min(max(np.random.randn()*log_freq_high[1] + log_freq_high[0],\
                                   log_freq_seq_array[0]),log_freq_seq_array[-1])
                    tone_kind.append('h')
                nearest_log_tone = log_freq_seq_array[np.argmin(np.abs(log_freq_seq_array - tone))]
                nearest_tone = freq_seq[np.argmin(np.abs(freq_seq - 10**nearest_log_tone))]        
            tones.append(nearest_tone)
        trial_tones.append(tones)
        dist_chosen.append(low_dist)
        kind_of_tones.append(tone_kind)
        
    return trial_tones, dist_chosen, kind_of_tones

In [63]:
# Following is for the testing part of the three tones expt.
testing_trials, dist_chosen, kind_of_tones = test_task(n_trials = 80, n_tones = 2)

background_tone_cnt = np.zeros((len(testing_trials),1))
for i_trial in range(len(testing_trials)):
    background_tone_cnt[i_trial] = kind_of_tones[i_trial].count('b')
num_difficult_tones = np.sum(background_tone_cnt > 1)

old_difficult_tones_idxs = np.where(background_tone_cnt > 1)[0][:]
old_simple_tones_idxs = np.where(background_tone_cnt < 2)[0][:]

pdb.set_trace()

if (np.diff(old_difficult_tones_idxs)<2).any():
    new_difficult_tones_idxs = \
    [2*i + x \
     for i, x \
     in enumerate(sorted(random.sample(range(2*num_difficult_tones), num_difficult_tones)))]
    new_simple_tones_idxs = np.delete(np.arange(len(testing_trials)),new_difficult_tones_idxs)
    new_testing_trials = np.zeros_like(testing_trials)   
    new_testing_trials[new_difficult_tones_idxs] = np.array(testing_trials)[old_difficult_tones_idxs]
    new_testing_trials[new_simple_tones_idxs] = np.array(testing_trials)[old_simple_tones_idxs]
    new_dist_chosen = np.zeros_like(dist_chosen)
    new_dist_chosen[new_difficult_tones_idxs] = np.array(dist_chosen)[old_difficult_tones_idxs]
    new_dist_chosen[new_simple_tones_idxs] = np.array(dist_chosen)[old_simple_tones_idxs]
    new_kind_of_tones = np.zeros_like(kind_of_tones)
    new_kind_of_tones[new_difficult_tones_idxs] = np.array(kind_of_tones)[old_difficult_tones_idxs]
    new_kind_of_tones[new_simple_tones_idxs] = np.array(kind_of_tones)[old_simple_tones_idxs]
    
    testing_trials = list(new_testing_trials)
    dist_chosen = list(new_dist_chosen)
    kind_of_tones = list(new_kind_of_tones)
    

--Return--
> <ipython-input-63-8de949d04d45>(12)<module>()->None
-> pdb.set_trace()
(Pdb) continue


In [64]:
print(sum(background_tone_cnt == 0), sum(background_tone_cnt == 1), sum(background_tone_cnt == 2))

no_background = np.where(background_tone_cnt==0)[0][:]
print(np.array(testing_trials)[no_background])

[40] [34] [6]
[[ 744 1066]
 [ 294  374]
 [ 603  767]
 [1034  535]
 [ 475  701]
 [1276  744]
 [ 199  421]
 [ 246  285]
 [ 277  322]
 [ 294  246]
 [ 225  285]
 [ 285  261]
 [ 182  261]
 [ 891  891]
 [ 225  363]
 [ 814  744]
 [ 660  722]
 [ 322  397]
 [ 322  171]
 [ 519  701]
 [ 269  238]
 [ 374  261]
 [ 269  246]
 [ 744  568]
 [ 193  269]
 [ 814  421]
 [ 814  839]
 [ 238  261]
 [ 188  374]
 [1396  701]
 [ 767  891]
 [ 622  790]
 [ 261  303]
 [ 374  489]
 [ 303  261]
 [ 269  303]
 [ 461  603]
 [ 285  261]
 [ 253  246]
 [ 535  603]]


In [20]:
"""
For two tones
Since tones are repeated, plotting only unique tones and mean(keys) pairs 
"""
unique_tones, unique_index, unique_count = np.unique(testing_trials, return_index = True, return_counts = True)
print(np.concatenate((np.expand_dims(unique_tones,1), np.expand_dims(unique_count,1)),axis=1))

[[ 100   58]
 [ 103    5]
 [ 106    6]
 [ 109    6]
 [ 112    2]
 [ 115    1]
 [ 118    7]
 [ 121    5]
 [ 124    2]
 [ 127    7]
 [ 130    3]
 [ 133    2]
 [ 139    6]
 [ 142    2]
 [ 145    6]
 [ 148    8]
 [ 151    7]
 [ 154    6]
 [ 157    4]
 [ 160    4]
 [ 163    7]
 [ 166    7]
 [ 169    3]
 [ 172    4]
 [ 175    6]
 [ 178    6]
 [ 181    2]
 [ 184    1]
 [ 187    2]
 [ 190    2]
 [ 193    4]
 [ 196    2]
 [ 199    3]
 [ 202    6]
 [ 205    9]
 [ 208    3]
 [ 211    6]
 [ 214    2]
 [ 217    2]
 [ 220    4]
 [ 223    3]
 [ 226    3]
 [ 229    9]
 [ 232   11]
 [ 235   12]
 [ 238    7]
 [ 241    4]
 [ 244   11]
 [ 247   10]
 [ 250   10]
 [ 253    7]
 [ 256    6]
 [ 259    9]
 [ 262    9]
 [ 265   11]
 [ 268    7]
 [ 271   12]
 [ 274   11]
 [ 277    9]
 [ 280   13]
 [ 283    3]
 [ 286    5]
 [ 289   10]
 [ 292    8]
 [ 295    7]
 [ 298    5]
 [ 301   11]
 [ 304    9]
 [ 307    6]
 [ 310    5]
 [ 313    7]
 [ 316   11]
 [ 319    2]
 [ 322    5]
 [ 325    2]
 [ 328    4]
 [ 331    9]

In [41]:
np.log10(1003/1000)*10, np.log10(20)/(np.log10(1003/1000)*10)

(0.01300933020418072, 100.00745428430113)

In [28]:
cnt_low = 0; cnt_high = 0
rows_csv_tt = []
for i_trial in range(len(testing_trials)):
    audio, sample_rate = generate_tone_cloud(testing_trials[i_trial])    
    if dist_chosen[i_trial]:
        save_wav(audio, sample_rate, file_name = \
                 "soundfiles_two_tones_additional_test_tones/test_wave_low_"+str(cnt_low)+".wav")
        rows_csv_tt.append(["test_wave_low_"+str(cnt_low)+".wav",'l',testing_trials[i_trial],\
                           kind_of_tones[i_trial]])
        cnt_low += 1  
    else:
        save_wav(audio, sample_rate, file_name = \
                 "soundfiles_two_tones_additional_test_tones/test_wave_high_"+str(cnt_high)+".wav")
        rows_csv_tt.append(["test_wave_high_"+str(cnt_high)+".wav",'h',testing_trials[i_trial],\
                           kind_of_tones[i_trial]])
        cnt_high += 1 

TypeError: only size-1 arrays can be converted to Python scalars

In [ ]:
# field names  
fields = ['Name', 'corrAns', 'Tones']  
    
# name of csv file  
filename = "test_two_tones_files_additional_tones.csv"
    
# writing to csv file  
with open(filename, 'w') as csvfile:  
    # creating a csv writer object  
    csvwriter = csv.writer(csvfile)          
    # writing the fields  
    csvwriter.writerow(fields)          
    # writing the data rows  
    csvwriter.writerows(rows_csv_tt) 